In [2]:
# Imports all the libraries required
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon

In [3]:
# Reads in the twitter data.
tweet_data_chunk = pd.read_json('geotagged_tweets_20160812-0912.jsons', lines=True, chunksize=1000)

In [4]:
print(tweet_data_chunk)

In [9]:
# Code from https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in tweet_data_chunk:  
    # perform data filtering 
    df = pd.read_json(chunk)
    print(df)

ValueError: Invalid file path or buffer object type: <class 'pandas.core.frame.DataFrame'>

In [ ]:
df_concat